# LangChain Prompt Templates
Greatly simplifies the task of creating maintaible prompts.

#### Google Colab
If you are running the code in Google colab, install the packages by uncommenting/running the cell below

* The API key file file will not be available
* You will be prompted to provide the Cohere API Token

Uncomment & run the code in the cell below:

In [1]:
# !pip install langchain -q

## 1. PromptTemplate

Use a formatted string with placeholders. The placeholders are referred to as inputs in the context of LangChain prompts

In [5]:
from langchain.prompts import PromptTemplate

# A prompt with two inputs {subject} and {grade}
instruction_template = """
You are a helpful {subject} teacher for grade {grade} class
"""

# create a prompt template 
prompt_template = PromptTemplate(
    template = instruction_template,
    input_variables = ["subject", "grade"]
)


In [8]:
# To create a prompt, you must supply value for all placeholders

prompt = prompt_template.format(subject="science", grade="6")

prompt

'\nYou are helpful science teacher for grade 6 class\n'

## 2. Partial prompts

https://python.langchain.com/docs/modules/model_io/prompts/partial

Sometimes you may not have all the input variables needed for a prompt. In those situations, you can use the *partial* prompt template to incrementally prepare the prompt (as variables become available).

### partial string
The input variables take the values as string with the call to partial function. The output is not a string but a *PromptTemplate*.

In [ ]:
## This will give an error
# prompt = prompt_template.format(subject="science")

In [10]:
# use *partial* instead of *format*
partial_prompt = prompt_template.partial(subject="science")

In [11]:
# At a later time provide the other inputs as they become availabe
prompt = partial_prompt.format(grade="6")
prompt

'\nYou are helpful science teacher for grade 6 class\n'

### partial function
Provide a reference to function instead of a string.

In [13]:
import random

# utility function for demo
def generate_random_number():
  """Generates a random number between start and end, inclusive."""
  return random.randint(1, 10)


7

In [40]:
# Set the 
partial_prompt = prompt_template.partial(grade=generate_random_number)

prompt = partial_prompt.format(subject="science")

prompt


<class 'langchain_core.prompts.prompt.PromptTemplate'>


## 3. FewShotPrompt

https://python.langchain.com/docs/modules/model_io/prompts/few_shot_examples
    
https://api.python.langchain.com/en/stable/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate



#### Prepare a template for examples

In [24]:
# sample examples to be used in the prompt
examples = [
    
    {
        "question": "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?",
        "answer": "Natalia sold 48/2 = <<48/2=24>>24 clips in May. Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May. #### 72"
    },
    {
        "question": "Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
        "answer": "Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute. Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10. #### 10"        
    },
    {
        "question": "Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?",
        "answer": "In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50. Betty's grandparents gave her 15 * 2 = $<<15*2=30>>30. This means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more. #### 5"
    }
]

# define the prompt template without examples
example_template = "question: {question}\nanswer:{answer}"

# template for the example
example_prompt_template = PromptTemplate(
    template = example_template,
    input_variables = ["question", "answer"]
)

# test the example prompt
# print(example_template.format(**examples[0]))

#### Prepare a template for the few shot prompt

In [26]:
from langchain.prompts import FewShotPromptTemplate


fewshot_template = FewShotPromptTemplate(
    # example prompt template
    example_prompt = example_prompt_template,
    
    # examples
    examples = examples,
    
    # A prompt template string to put after the examples
    suffix ="question: {input}",
    
    input_variables = ["input"]
)

In [29]:
input = "Albert is wondering how much pizza he can eat in one day. He buys 2 large pizzas and 2 small pizzas. A large pizza has 16 slices and a small pizza has 8 slices. If he eats it all, how many pieces does he eat that day?"

prompt = fewshot_template.format(input=input)

print(prompt)

question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
answer:Natalia sold 48/2 = <<48/2=24>>24 clips in May. Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May. #### 72

question: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
answer:Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute. Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10. #### 10

question: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
answer:In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50. Betty's grandparents gave her 15 * 2 = $<<15*2=30>>30. This means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30

## 4. ExampleSelector

https://python.langchain.com/docs/modules/model_io/prompts/few_shot_examples#using-an-example-selector
    
https://api.python.langchain.com/en/stable/core_api_reference.html#module-langchain_core.example_selectors

https://python.langchain.com/docs/modules/model_io/prompts/example_selector_types/length_based

In [37]:
# Change the length to see the difference in the prompt 
# You will always see a full example!!

from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = example_prompt_template,
    max_length=100,
    
    # you can provide your own function for get the length of example
    # get_text_length = your func reference
)

In [38]:
fewshot_template = FewShotPromptTemplate(
    # example prompt template
    example_prompt = example_prompt_template,
    
    # examples - CANNOT provide examples if example_selector is provided
    # examples = examples,
    
    # examples selector
    example_selector = example_selector,
    
    # A prompt template string to put after the examples
    suffix ="question: {input}",
    
    input_variables = ["input"]
)

In [39]:
input = "Albert is wondering how much pizza he can eat in one day. He buys 2 large pizzas and 2 small pizzas. A large pizza has 16 slices and a small pizza has 8 slices. If he eats it all, how many pieces does he eat that day?"

prompt = fewshot_template.format(input=input)

print(prompt)

question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
answer:Natalia sold 48/2 = <<48/2=24>>24 clips in May. Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May. #### 72

question: Albert is wondering how much pizza he can eat in one day. He buys 2 large pizzas and 2 small pizzas. A large pizza has 16 slices and a small pizza has 8 slices. If he eats it all, how many pieces does he eat that day?
